In [1]:
import requests
import pandas as pd
import csv

# 1. Obtener lista de países y territorios del Banco Mundial
resp = requests.get("https://api.worldbank.org/v2/country",params={"format": "json", "per_page": 400})

data = resp.json()
paises = data[1]

# Crear tabla con datos básicos de cada país (DataFrame)
df_countries = pd.DataFrame([
    {
        "name": p["name"],
        "iso3Code": p["id"],
        "region": p["region"]["value"],
        "adminregion": p["adminregion"]["value"]
    }
    for p in paises
])

# Flitro para quitar agregados ( adminregion = "" )
df_countries = df_countries[df_countries["adminregion"] != ""].reset_index(drop=True)

# *************************************
# **********  IMPORTANTE *************
# *************************************
#Comenta y descomenta según convenga para descargar el .csv

# descargar = True
descargar = False

#Usamos funcion para encaptular toda la logica de descarga
def logica_descarga(iso3, indicator_code, start_year=1980, end_year=2023):
    
    #URL para el get de un pais
    url = f"https://api.worldbank.org/v2/country/{iso3}/indicator/{indicator_code}"
    # Los parametros ponemos 20000 para coger todas las pag 
    params = {"format": "json","per_page": 20000,"date": f"{start_year}:{end_year}"}

    #Hacemos la peticion a la API
    resp = requests.get(url, params=params)
    # Respuesta = json
    data = resp.json()

    #convertimos la lista a diccionaro año:valor para acceder mas rapido
    series = {}
    for entry in data[1]:
        year = entry["date"]
        value = entry["value"]
        series[year] = value
    
    return series


if descargar:
    #Los codigos que necesitamos
    esperanza_vida = "SP.DYN.LE00.IN"
    pib = "NY.GDP.PCAP.CD"

    with open("data/datos.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerow(["country", "iso3", "region", "year", "life_expectancy", "gdp_per_capita"]) # hombre de las columnas
        
        #Recorremos cada pais del DataFrame
        for i, row in df_countries.iterrows():
            iso3 = row["iso3Code"]
            name = row["name"]
            region = row["region"]

            print(f"Descargando {name}")

            esperanza_series = logica_descarga(iso3, esperanza_vida)
            pib_series = logica_descarga(iso3, pib)

            # Recorremos cada año y hacemos una fila por año
            for year in range(1980, 2023 + 1):
                y = str(year)
                writer.writerow([
                    name,
                    iso3,
                    region,
                    y,
                    esperanza_series.get(y),
                    pib_series.get(y)
                ])

            print(f"{name} Descargado")
else:
    print("Comprueba si la tienes descargada si no, descarga : True ")

Comprueba si la tienes descargada si no, descarga : True 
